In [ ]:
import os
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
import string
import re
import pickle
import gzip

In [22]:
words = word_tokenize('1,000')
for word in words:
    print(re.split('[,\']', mystring))
# wordsFiltered = list(filter(lambda token: token not in string.punctuation, words))
# print(wordsFiltered)

['10.200.200.100', '1', '000', 'hello', 'don', 't']


In [ ]:
sen = 'rodwin:74996,4,23,66,68,87;'
print(re.sub('^.*:', '', sen))

In [ ]:
stopWords = []

In [ ]:
catalogs = []

In [ ]:
def writeToFile(path, invertedList, file_num, catalog_list):
    catalog = dict()
    filename = "partial_inverted_list" + str(file_num) + ".txt"
    with open(os.path.join(path, filename), 'a') as f:
        for term in invertedList.keys():
            startOffset = f.tell()
            line = str(term) + ":"
            for docid in invertedList[term]:
                if line!= str(term) + ":":
                    line+= ','
                line +=  str(doc_mapping[docid]) + "," + str(invertedList[term][docid]['tf'])+ ',' +','.join(str(v) for v in invertedList[term][docid]['positions'])
            f.write(line + ";")
            endOffset = f.tell()
            catalog[term] = [startOffset, endOffset - startOffset]
        catalog_list.append(catalog)

In [ ]:
with open("D:\\CS 6200\\AP_DATA\\stoplist.txt") as f:
    for line in f:
        line = line.rstrip()
        stopWords.append(line.rstrip())

In [ ]:
doc_mapping = dict()

In [ ]:
def get_docid_mapping(id_mapping_file):
    path = 'D:\\CS 6200\\AP_DATA'
    docid_to_num = dict()
    with open(os.path.join(path, id_mapping_file), 'r') as f:
        for line in f:
            doc_mapping = line.split()
            docid_to_num[doc_mapping[0]] = doc_mapping[1]
    return docid_to_num

In [ ]:
doc_mapping = get_docid_mapping('docmapping.txt')

In [ ]:
print(doc_mapping)

In [ ]:
doc_lengths = dict()

In [ ]:
def parse(u):

    buffer = ""
    token_dict = dict()
    num_of_docs = 0
    writes = 0
    token_position = 0
    ps = PorterStemmer()
    
    try:
        for filename in os.listdir(u):
            with open(os.path.join(path, filename)) as f:
                buffer = ""
                for line in f:
                    if "</DOC>" in line:
                        num_of_docs += 1
                        docid = re.findall(r"<DOCNO> (.*?) </DOCNO>", buffer)
                        contents = re.findall(r"<TEXT> (.*?) </TEXT>", buffer, re.VERBOSE|re.MULTILINE|re.DOTALL)
                        token_position = 0
                        contents[0] = contents[0].replace('<TEXT>',' ')
                        contents[0] = contents[0].replace('</TEXT>',' ')
                        contents[0] = contents[0].replace('-',' ')
                        contents[0] = contents[0].replace('``', ' ')
                        tokens = word_tokenize(contents[0].lower())
                        wordsFiltered = []

                        for w in tokens:
                            if w not in stopWords:
                                w = ps.stem(w)
                                wordsFiltered.append(w)

                        wordsFiltered = list(filter(lambda token: token not in string.punctuation, wordsFiltered))
                        for word in wordsFiltered:
                            token_position += 1
                            if word not in token_dict:
                                token_dict[word] = {}
                                token_dict[word][docid[0]] = {}
                                token_dict[word][docid[0]]['tf'] = 1
                                token_dict[word][docid[0]]['positions'] = []
                            elif word in token_dict:
                                if docid[0] in token_dict[word]:
                                    token_dict[word][docid[0]]['tf']  += 1
                                else:
                                    token_dict[word][docid[0]] = {}
                                    token_dict[word][docid[0]]['positions'] = []
                                    token_dict[word][docid[0]]['tf'] = 1

                            token_dict[word][docid[0]]['positions'].append(token_position)
                        doc_lengths[num_of_docs] = token_position
                        buffer = ""


                        if num_of_docs % 1000 == 0 or num_of_docs == 84678:
                            print(num_of_docs)
                            writes += 1
                            writeToFile('D:\\CS 6200\\Assignment 2\\stemmed', token_dict, writes, catalogs)
                            token_dict = dict()
                    if num_of_docs == 84678:
                        break

                    line = line.replace('\n',' ')
                    buffer = buffer + line
                    
       
    except Exception as ex:
        print('Exception while parsing')
        print(str(ex))
    # finally:
    #     return token_dict

In [ ]:
 path = "D:\\CS 6200\\AP_DATA\\ap89_collection"

In [ ]:
parse(path)

In [ ]:
print(doc_lengths[84631])

In [ ]:
with open('D:\\CS 6200\\Assignment 2\\stemmed\\doc_lengths.pickle', 'wb') as handle:
    pickle.dump(doc_lengths, handle)

In [ ]:
print(len(catalogs))

In [ ]:
print(catalogs[0])

In [ ]:
with open('D:\\CS 6200\\Assignment 2\\stemmed\\partial_inverted_list1.txt', 'r') as f:
    f.seek(1837467)
    print(f.read(19))

In [ ]:
def pickle_catalogs(catalog_list, path):  
    for i in range(0, len(catalog_list)):
        filename = 'catalog' + str(i+1) + '.pickle'
        with open(os.path.join(path, filename), 'wb') as handle:
            pickle.dump(catalogs[i], handle)


In [ ]:
pickle_catalogs(catalogs, 'D:\\CS 6200\\Assignment 2\\stemmed')

In [ ]:
def merge_partial_lists(c1, c2, inverted_index_list, outFileName, catalog_name):
    merged_catalog = dict()
    file1 = open(inverted_index_list[0], 'r')
    file2 = open(inverted_index_list[1], 'r')
    path = 'D:\\CS 6200\\Assignment 2\\stemmed'
    outFile = open(os.path.join(path, outFileName), 'a')

    for term in c1:
        if term not in merged_catalog:
            if term not in c2:
                file1.seek(c1[term][0])
                line = file1.read(c1[term][1])
                startOffset = outFile.tell()
                outFile.write(line)
                endOffset = outFile.tell()
                merged_catalog[term] = [startOffset, endOffset-startOffset]
            elif term in c2:
                file1.seek(c1[term][0])
                line = file1.read(c1[term][1]-1) + ","
                file2.seek(c2[term][0])
                file2_line = file2.read(c2[term][1])
                file2_line = re.sub('^.*:', '', file2_line)
                outLine = line + file2_line
                startOffset = outFile.tell()
                outFile.write(outLine)
                endOffset = outFile.tell()
                merged_catalog[term] = [startOffset, endOffset-startOffset]

    for term in c2:
        if term not in merged_catalog:
            file2.seek(c2[term][0])
            line = file2.read(c2[term][1])
            startOffset = outFile.tell()
            outFile.write(line)
            endOffset = outFile.tell()
            merged_catalog[term] = [startOffset, endOffset-startOffset]

    outFile.close()
    
    #return merged_catalog
    with open(os.path.join(path, catalog_name), 'wb') as handle:
            pickle.dump(merged_catalog, handle)
            

In [ ]:
def unpickle_catalog(catalog_name):
    path = 'D:\\CS 6200\\Assignment 2\\stemmed'
    with open(os.path.join(path, catalog_name), 'rb') as handle:
        b = pickle.load(handle)
    return b

In [ ]:
def get_full_file_path(file_name):
    path = 'D:\\CS 6200\\Assignment 2\\stemmed'
    return os.path.join(path, file_name)

In [ ]:
iilist = ['D:\\CS 6200\\Assignment 2\\stemmed\\partial_inverted_list1.txt', 'D:\\CS 6200\\Assignment 2\\stemmed\\partial_inverted_list2.txt']

In [ ]:
joined_catalog = merge_partial_lists(catalogs[0], catalogs[1], iilist, 'l2_merged_index.txt', 'l2_catalog.pickle')

In [ ]:
for i in range(3, 86): 
    c1_file_name = 'l' + str(i-1) + '_catalog.pickle'
    c2_file_name = 'catalog' + str(i) + '.pickle'
    print
    c1 = unpickle_catalog(c1_file_name) 
    c2 = unpickle_catalog(c2_file_name)  
    partial_list_name1 = 'l' + str(i-1) + '_merged_index.txt'
    partial_list_name2 = 'partial_inverted_list' + str(i) + '.txt'
    ii_list = [get_full_file_path(partial_list_name1), get_full_file_path(partial_list_name2)]
    outFile = 'l' + str(i) + '_merged_index.txt'
    catalog_name = 'l' + str(i) + '_catalog.pickle'
    print(ii_list, '\n', c1_file_name, '\n', c2_file_name, '\n', catalog_name, '\n', outFile)
    merge_partial_lists(c1, c2, ii_list, outFile, catalog_name)

In [ ]:
print(len(list(unpickle_catalog('l85_catalog.pickle').keys())))

In [ ]:
def move():
    icPath = 'D:\\CS 6200\\Assignment 2\\stemmed'
    with open(os.path.join(icPath, 'l85_merged_index.txt'), 'r') as fin:
        d = fin.read()
    fin.close()
    b = d.encode()
    with gzip.open(os.path.join(icPath, 'file.txt.gz'), 'w+') as fou:
        fou.write(b)
    fou.close()

In [ ]:
move()